# this should be in backend
testing here for convenience

In [8]:
import os
import json
import pandas as pd

import torch
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util

import mysql.connector
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine
from sqlalchemy import text as sqltext
load_dotenv(find_dotenv())

from svs import SVS
from queryOpenAI import run_engine

In [2]:
host = os.getenv("DB_HOST")
user = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

In [3]:
# preloading slow things
model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')
f = open('../models/mimic-iii/qqdict.json')
qqdict = json.load(f)
corpus_embedding = torch.tensor(pd.read_csv("../models/mimic-iii/corpus.csv").values).to(torch.float32)

client = OpenAI(api_key=os.getenv("OAI_KEY"))

/Users/isaac/Library/Python/3.9/lib/python/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = create_engine(conn_string, echo=True)
conn = engine.connect()

2024-02-11 04:57:16,897 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-02-11 04:57:16,898 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 04:57:17,070 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-02-11 04:57:17,071 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 04:57:17,143 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-02-11 04:57:17,143 INFO sqlalchemy.engine.Engine [raw sql] {}


In [14]:
question = "what is the change in the weight of patient 6215 from the second value measured on the first hospital visit compared to the first value measured on the first hospital visit?"

In [16]:
def backend(question):
    qqlist = [qqdict[str(i)] for i in SVS(question, k=15, corpus=corpus_embedding, model=model)]

    relevant_qtext = "# Q: \t" + "\n# Q: \t".join(["\n# A: \t".join(rel) for rel in qqlist])
    test_prompt = open('../models/mimic-iii/codex_apidoc.txt').read()

    result = run_engine(question, test_prompt, relevant_qtext)

    print(result)
    print()
    return result
    
backend(question)

# Q: 	how much does patient 15794 change in weight last measured on the current hospital visit compared to the second to last value measured on the current hospital visit?
# A: 	select ( select chartevents.valuenum from chartevents where chartevents.icustay_id in ( select icustays.icustay_id from icustays where icustays.hadm_id in ( select admissions.hadm_id from admissions where admissions.subject_id = 15794 and admissions.dischtime is null ) ) and chartevents.itemid in ( select d_items.itemid from d_items where d_items.label = 'admit wt' and d_items.linksto = 'chartevents' ) order by chartevents.charttime desc limit 1 ) - ( select chartevents.valuenum from chartevents where chartevents.icustay_id in ( select icustays.icustay_id from icustays where icustays.hadm_id in ( select admissions.hadm_id from admissions where admissions.subject_id = 15794 and admissions.dischtime is null ) ) and chartevents.itemid in ( select d_items.itemid from d_items where d_items.label = 'admit wt' and d_i

"SELECT( SELECT CHARTEVENTS.VALUENUM \n  FROM CHARTEVENTS \n  WHERE CHARTEVENTS.ICUSTAY_ID IN \n    ( SELECT ICUSTAYS.ICUSTAY_ID \n      FROM ICUSTAYS \n      WHERE ICUSTAYS.HADM_ID IN \n        ( SELECT ADMISSIONS.HADM_ID \n          FROM ADMISSIONS \n          WHERE ADMISSIONS.SUBJECT_ID = 6215 \n          AND ADMISSIONS.DISCHTIME IS NOT NULL \n          ORDER BY ADMISSIONS.ADMITTIME ASC LIMIT 1 \n        ) \n    ) \n  AND CHARTEVENTS.ITEMID IN \n    ( SELECT D_ITEMS.ITEMID \n      FROM D_ITEMS \n      WHERE D_ITEMS.LABEL = 'ADMIT WT' \n      AND D_ITEMS.LINKSTO = 'CHARTEVENTS' \n    ) \n  ORDER BY CHARTEVENTS.CHARTTIME ASC LIMIT 1 OFFSET 1 \n) - \n( SELECT CHARTEVENTS.VALUENUM \n  FROM CHARTEVENTS \n  WHERE CHARTEVENTS.ICUSTAY_ID IN \n    ( SELECT ICUSTAYS.ICUSTAY_ID \n      FROM ICUSTAYS \n      WHERE ICUSTAYS.HADM_ID IN \n        ( SELECT ADMISSIONS.HADM_ID \n          FROM ADMISSIONS \n          WHERE ADMISSIONS.SUBJECT_ID = 6215 \n          AND ADMISSIONS.DISCHTIME IS NOT NULL \

In [20]:
test_prompt = open('../models/mimic-iii/codex_apidoc.txt').read()

In [5]:
print(question)
print()
print("SELECT COUNT(*)>0 \nFROM PRESCRIPTIONS \nWHERE PRESCRIPTIONS.HADM_ID IN \n( \n    SELECT ICUSTAYS.HADM_ID \n    FROM ICUSTAYS \n    WHERE ICUSTAYS.SUBJECT_ID = 22973 \n    ORDER BY ICUSTAYS.INTIME DESC \n    LIMIT 1 \n) \nAND PRESCRIPTIONS.DRUG = 'INSULIN';")

Was patient 22973 prescribed insulin the last time they were at the ICU?

SELECT COUNT(*)>0 
FROM PRESCRIPTIONS 
WHERE PRESCRIPTIONS.HADM_ID IN 
( 
    SELECT ICUSTAYS.HADM_ID 
    FROM ICUSTAYS 
    WHERE ICUSTAYS.SUBJECT_ID = 22973 
    ORDER BY ICUSTAYS.INTIME DESC 
    LIMIT 1 
) 
AND PRESCRIPTIONS.DRUG = 'INSULIN';


In [17]:
text = '''SELECT( SELECT CHARTEVENTS.VALUENUM \n  FROM CHARTEVENTS \n  WHERE CHARTEVENTS.ICUSTAY_ID IN \n    ( SELECT ICUSTAYS.ICUSTAY_ID \n      FROM ICUSTAYS \n      WHERE ICUSTAYS.HADM_ID IN \n        ( SELECT ADMISSIONS.HADM_ID \n          FROM ADMISSIONS \n          WHERE ADMISSIONS.SUBJECT_ID = 6215 \n          AND ADMISSIONS.DISCHTIME IS NOT NULL \n          ORDER BY ADMISSIONS.ADMITTIME ASC LIMIT 1 \n        ) \n    ) \n  AND CHARTEVENTS.ITEMID IN \n    ( SELECT D_ITEMS.ITEMID \n      FROM D_ITEMS \n      WHERE D_ITEMS.LABEL = 'ADMIT WT' \n      AND D_ITEMS.LINKSTO = 'CHARTEVENTS' \n    ) \n  ORDER BY CHARTEVENTS.CHARTTIME ASC LIMIT 1 OFFSET 1 \n) - \n( SELECT CHARTEVENTS.VALUENUM \n  FROM CHARTEVENTS \n  WHERE CHARTEVENTS.ICUSTAY_ID IN \n    ( SELECT ICUSTAYS.ICUSTAY_ID \n      FROM ICUSTAYS \n      WHERE ICUSTAYS.HADM_ID IN \n        ( SELECT ADMISSIONS.HADM_ID \n          FROM ADMISSIONS \n          WHERE ADMISSIONS.SUBJECT_ID = 6215 \n          AND ADMISSIONS.DISCHTIME IS NOT NULL \n          ORDER BY ADMISSIONS.ADMITTIME ASC LIMIT 1 \n        ) \n    ) \n  AND CHARTEVENTS.ITEMID IN \n    ( SELECT D_ITEMS.ITEMID \n      FROM D_ITEMS \n      WHERE D_ITEMS.LABEL = 'ADMIT WT' \n      AND D_ITEMS.LINKSTO = 'CHARTEVENTS' \n    ) \n  ORDER BY CHARTEVENTS.CHARTTIME ASC LIMIT 1 \n);'''

In [19]:
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = create_engine(conn_string, echo=True)
connection = engine.connect()

try:
    result = connection.execute(sqltext(text)).fetchall()
except Exception as e:
    print(e)

2024-02-11 05:40:24,382 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-02-11 05:40:24,386 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 05:40:24,617 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-02-11 05:40:24,618 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 05:40:24,688 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-02-11 05:40:24,689 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 05:40:24,803 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 05:40:24,804 INFO sqlalchemy.engine.Engine SELECT( SELECT CHARTEVENTS.VALUENUM 
  FROM CHARTEVENTS 
  WHERE CHARTEVENTS.ICUSTAY_ID IN 
    ( SELECT ICUSTAYS.ICUSTAY_ID 
      FROM ICUSTAYS 
      WHERE ICUSTAYS.HADM_ID IN 
        ( SELECT ADMISSIONS.HADM_ID 
          FROM ADMISSIONS 
          WHERE ADMISSIONS.SUBJECT_ID = 6215 
          AND ADMISSIONS.DISCHTIME IS NOT NULL 
          ORDER BY ADMISSIONS.ADMITTIME ASC LIMIT 1 
        ) 
    ) 
  AND CHARTEVENTS.ITEMID IN 
    ( 